<a href="https://colab.research.google.com/github/Sangram-1409/AgriSmart/blob/main/Real_time_Cryptocurrency_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The purpose of this script is to fetch and display real-time cryptocurrency prices.
# It uses a public API (CoinGecko) and updates the prices at a regular interval.

# This script can be a foundation for more complex projects, such as:
# 1. Integrating with a carbon token platform to track the value of your earned carbon credits.
# 2. Creating a simple portfolio tracker.
# 3. Building a backend for a web-based price chart.

import requests
import time
import os

# --- Configuration ---
# Set the API endpoint for CoinGecko. This API is free and doesn't require an API key for this use case.
API_URL = "https://api.coingecko.com/api/v3/simple/price"

# Define the cryptocurrencies you want to track and the currencies for the price display.
# The `ids` and `vs_currencies` must match the API's supported list.
# Bitcoin and Ethereum are standard for demonstration.
CRYPTO_IDS = "bitcoin,ethereum"
VS_CURRENCIES = "usd"

# Set the update interval in seconds.
UPDATE_INTERVAL = 10

def clear_screen():
    """Clears the console screen for a clean, real-time display."""
    # os.name is 'nt' for Windows, 'posix' for Linux/macOS
    os.system('cls' if os.name == 'nt' else 'clear')

def fetch_crypto_prices(ids, vs_currencies):
    """
    Fetches cryptocurrency prices from the CoinGecko API.

    Args:
        ids (str): A comma-separated string of cryptocurrency IDs.
        vs_currencies (str): A comma-separated string of price currencies.

    Returns:
        dict: A dictionary of price data or None if the request fails.
    """
    try:
        # Build the request parameters.
        params = {
            "ids": ids,
            "vs_currencies": vs_currencies
        }

        # Make the GET request to the API.
        response = requests.get(API_URL, params=params)

        # Raise an exception for bad status codes (4xx or 5xx).
        response.raise_for_status()

        # Return the JSON data from the response.
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def main():
    """Main function to run the real-time crypto tracker."""
    print("Starting real-time crypto tracker...")
    print("Press Ctrl+C to stop.\n")

    try:
        while True:
            # Fetch the latest prices.
            prices = fetch_crypto_prices(CRYPTO_IDS, VS_CURRENCIES)

            if prices:
                # Clear the screen to display the updated data.
                clear_screen()

                # Print a header with the current time.
                current_time = time.strftime("%Y-%m-%d %H:%M:%S")
                print(f"--- Cryptocurrency Prices ({current_time}) ---")

                # Iterate through the fetched prices and print them.
                for crypto_id, data in prices.items():
                    # Capitalize the cryptocurrency name for display.
                    crypto_name = crypto_id.replace('-', ' ').title()
                    price = data.get(VS_CURRENCIES)

                    if price is not None:
                        # Format the output for readability.
                        print(f"💰 {crypto_name:<10}: ${price:,.2f}")
                    else:
                        print(f"❌ {crypto_name:<10}: Price not available.")

                print("\n-------------------------------------------")

            # Wait for the specified interval before the next update.
            time.sleep(UPDATE_INTERVAL)

    except KeyboardInterrupt:
        # Handle the user pressing Ctrl+C gracefully.
        print("\nTracker stopped by user. Goodbye!")
    except Exception as e:
        # Handle any other unexpected errors.
        print(f"An unexpected error occurred: {e}")

# Ensure the main function runs only when the script is executed directly.
if __name__ == "__main__":
    main()

Starting real-time crypto tracker...
Press Ctrl+C to stop.

--- Cryptocurrency Prices (2025-08-20 17:12:28) ---
💰 Bitcoin   : $114,055.00
💰 Ethereum  : $4,314.94

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:12:39) ---
💰 Bitcoin   : $114,043.00
💰 Ethereum  : $4,318.73

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:12:49) ---
💰 Bitcoin   : $114,043.00
💰 Ethereum  : $4,318.73

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:12:59) ---
💰 Bitcoin   : $114,055.00
💰 Ethereum  : $4,315.07

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:13:10) ---
💰 Bitcoin   : $114,043.00
💰 Ethereum  : $4,318.73

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:13:20) ---
💰 Bitcoin   : $114,043.00
💰 Ethereum  : $4,318.73

-------------------------------------------
--- Cryptocurrency Prices (2025-08-20 17:13:31) ---
💰 Bitc